# Deploy a pretrained sentiment model from Watson NLP as a Python function

This notebook shows the steps needed to create and deploy a Python function that predicts sentiment in Watson Machine Learning. the example creates an _online deployment_ of the Python function. This way, the sentiment analysis can be invoked via a web service in applications.

## Prerequisites 

Before running this notebook: Create a new deployment space, or identify an existing deployment space to use for this example

## Initialize the Watson Machine Learning client

In [ ]:
CPD_URL = '<CPD PLATFORM URL>'

In [ ]:
from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space()

In [ ]:
wml_credentials = {
    "url": CPD_URL,
    "token": wslib.auth.get_current_token(),
    "instance_id": "wml_local",
    "version" : "5.0"
}

In [ ]:
from ibm_watson_machine_learning import APIClient
client = APIClient(wml_credentials)

In [ ]:
client.spaces.list()

### Add the `ID` of the space to use for deploying the function

In [ ]:
client.set.default_space('<SPACE ID>')

## Create a Python function that detects sentiment

This function is using the targeted sentiment block `targets-sentiment_transformer-workflow_multilingual_slate.153m.distilled-cpu` provided by Watson NLP.

In [ ]:
def detect_sentiment():
    import watson_nlp
    
    sentiment_model = targets_sentiment_model = watson_nlp.load('targets-sentiment_transformer-workflow_multilingual_slate.153m.distilled-cpu')
    
    def score(input):
        scoring_prediction_out = []
        for input_data_row in input["input_data"][0]["values"]: 
            scoring_prediction_row = []
            for input_data in input_data_row:            
                targets_sentiments = targets_sentiment_model.run(input_data)
                scoring_prediction_row.append(targets_sentiments.to_dict())
                
            scoring_prediction_out.append(scoring_prediction_row)
            
        # Score using the pre-defined model
        scoring_response = {
            'predictions': [{'fields': ['nlp_prediction'], 
                             'values': scoring_prediction_out
                            }]
        }

        return scoring_response
    
    return score

## Save the Python function to the Model repository

In [ ]:
pyfunc_swspec_id = client.software_specifications.get_uid_by_name("runtime-24.1-py3.11")

meta_data = {
    client.repository.FunctionMetaNames.NAME: 'sentiment_function',
    client.repository.FunctionMetaNames.DESCRIPTION: 'sentiment_function',
    client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID: pyfunc_swspec_id
}


sentiment_function_details = client.repository.store_function(meta_props=meta_data, function=detect_sentiment)

## Create an Online Deployment

In [ ]:
sentiment_function_uid = client.repository.get_function_uid(sentiment_function_details)

meta_props = {
   client.deployments.ConfigurationMetaNames.NAME: "nlp-sentiment-deploy",
   client.deployments.ConfigurationMetaNames.DESCRIPTION: "nlp-sentiment-deploy",
   client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: { 'name': 'S'},  
   client.deployments.ConfigurationMetaNames.ONLINE: {   }
}

sentiment_deployment_details = client.deployments.create(sentiment_function_uid, meta_props=meta_props)
sentiment_deployment_id = client.deployments.get_uid(sentiment_deployment_details)

.

.

.

.

.

...

.

.

.

.


ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='4b6b02b2-d256-45ae-b683-c5d458fca6c4'
------------------------------------------------------------------------------------------------


{'entity': {'asset': {'id': '49dbf26f-eb7f-47ad-84d3-cb4101d4e359'}, 'custom': {}, 'deployed_asset_type': 'function', 'description': 'nlp-sentiment-deploy', 'hardware_spec': {'id': 'e7ed1d6c-2e89-42d7-aed5-863b972c1d2b', 'name': 'S', 'num_nodes': 1}, 'name': 'nlp-sentiment-deploy', 'online': {}, 'space_id': '4a2a89fb-9bd4-4e27-9301-372a71d1872f', 'status': {'inference': [{'url': 'https://cpd-dev.apps.midgard.cp.fyre.ibm.com/ml/v4/deployments/4b6b02b2-d256-45ae-b683-c5d458fca6c4/predictions'}], 'online_url': {'url': 'https://cpd-dev.apps.midgard.cp.fyre.ibm.com/ml/v4/deployments/4b6b02b2-d256-45ae-b683-c5d458fca6c4/predictions'}, 'serving_urls': ['https://cpd-dev.apps.midgard.cp.fyre.ibm.co

## Predict sentiment of sample data

In [ ]:
scoring_payload = {
   client.deployments.ScoringMetaNames.INPUT_DATA: [{  
        'fields': ["Text"],
        'values': [["The room is nice, but the price is too expensive. The window frame is made of wood."],
                   ["The check-in was horrible."],
                  ]
   }]
}

In [ ]:
predictions = client.deployments.score(sentiment_deployment_id, scoring_payload)
predictions

## Clean up: Delete the deployment and the function

In [ ]:
client.deployments.delete(sentiment_deployment_id)

In [ ]:
client.repository.delete(sentiment_function_uid)